In [ ]:
randomly generate a scale-free graph.  
then generate a random gaussian noise.  
then add bernoulli dropout.  

In [1]:
import networkx as nx
import scipy
import scipy.linalg as la
import numpy as np
import torch
# from gnn_magic import PolyBasis, LapPolyLayer, LapPolyNN, LinearLayer
# from train import make_dataloader, train
# from utils import lib_size_normalize, log_transform
# import matplotlib.pyplot as plt

def generate_gaussian(spec, V, n):
    y_sample = np.random.normal(0, 1, (n, len(spec)))
    # yf = spec_pow_n(spec, n_pow) * y_sample
    yf = spec * y_sample
    y = V @ yf.T
    return y.T
def f_drop(x, mean, scale=1.):
    return sigmoid((x - mean) * scale)
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [2]:
g = nx.scale_free_graph(1000)
N = g.number_of_nodes()
A = nx.adjacency_matrix(g)
A = (A + A.T) / 2
A += scipy.sparse.diags(np.ones(N)) * 1
D = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten())
Dinv = scipy.sparse.diags(1 / np.array(A.sum(axis=1)).flatten())
P = A @ Dinv
D_invhf = scipy.sparse.diags(1/np.sqrt(np.array(A.sum(axis=1)).flatten()))
L = D_invhf @ (D - A) @ D_invhf
Ld = L.toarray()
spec, V = la.eigh(Ld)
means = np.random.rand(1, N) * 20
n = 10000
z = torch.tensor(generate_gaussian(spec, V, n)) * 2 + means
z -= z.min()
p_drops = f_drop(z, 15., .2)
d = torch.bernoulli(p_drops)
x = d * z

In [3]:
# plt.hist(p_drops.flatten(), bins=100)
# plt.show()

In [4]:
# plt.hist(x.flatten(), bins=100)
# plt.show()

In [5]:
# plt.hist(z.flatten(), bins=100)
# plt.show()

In [6]:
# plt.scatter(z, x, s=.1, alpha=0.5)
# plt.show()

In [7]:
torch.save(x.float(), 'data/x.pt')
torch.save(z.float(), 'data/z.pt')